### Asynchronous `target` with `nowait` and `depend` Clauses

More details on dependences can be found in  task_depend , Task  Dependences. In this example, there are three flow dependences.  In the first two dependences the target task does not execute until the preceding explicit tasks have finished.   These  dependences are produced by arrays  _v1_  and  _v2_   with the `out` dependence type in the first two tasks, and the `in` dependence type in the target task.

The last dependence is produced by array  _p_   with the `out` dependence type in the target task, and the `in` dependence type in the last task.  The last task does not execute until the target task finishes.

The `nowait` clause on the `target` construct creates a deferrable  _target task_ , allowing the encountering task to continue execution without waiting for the completion of the  _target task_ .

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name: target.4c
* type: C
* version: omp_4.5
*/

extern void init(  float*, int);
extern void output(float*, int);

void vec_mult(int N)
{
   int i;
   float p[N], v1[N], v2[N];

   #pragma omp parallel num_threads(2)
   {
      #pragma omp single
      {
         #pragma omp task depend(out:v1)
         init(v1, N);

         #pragma omp task depend(out:v2)
         init(v2, N);

         #pragma omp target nowait depend(in:v1,v2) depend(out:p) \
                                      map(to:v1,v2) map( from: p)
         #pragma omp parallel for private(i)
         for (i=0; i<N; i++)
            p[i] = v1[i] * v2[i];

         #pragma omp task depend(in:p)
         output(p, N);
      }
   }
}



In [ ]:

! name: target.4f90
! type: F-free
! version: omp_4.5

subroutine vec_mult(N)
   implicit none
   integer           :: i, N
   real, allocatable :: p(:), v1(:), v2(:)
   allocate( p(N), v1(N), v2(N) )

   !$omp parallel num_threads(2)

      !$omp single

         !$omp task depend(out:v1)
         call init(v1, N)
         !$omp end task

         !$omp task depend(out:v2)
         call init(v2, N)
         !$omp end task

         !$omp target nowait depend(in:v1,v2) depend(out:p) &
         !$omp&                 map(to:v1,v2)  map(from: p)
         !$omp parallel do
         do i=1,N
            p(i) = v1(i) * v2(i)
         end do
         !$omp end target


         !$omp task depend(in:p)
         call output(p, N)
         !$omp end task

     !$omp end single
   !$omp end parallel

   deallocate( p, v1, v2 )

end subroutine

